# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint croix,17.7397,-64.7388,301.40,79,40,9.26,VI,1702323896
1,1,upington,-28.4478,21.2561,300.36,17,0,2.57,ZA,1702323897
2,2,el fasher,13.6279,25.3494,297.12,14,100,6.75,SD,1702323897
3,3,tofol,5.3248,163.0078,300.84,82,100,6.24,FM,1702323732
4,4,reggane,26.7158,0.1714,292.16,30,12,5.16,DZ,1702323897


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 900,
    frame_height = 400,
    size = "Humidity",
    scale = 1,
    hover_cols = ["City", "Humidity"])

# Display the map
cities_map

:Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[city_data_df["Max Temp"] < 294]
# Drop any rows with null values
ideal_weather_df_complete = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df_complete


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,reggane,26.7158,0.1714,292.16,30,12,5.16,DZ,1702323897
6,6,qaqortoq,60.7167,-46.0333,271.19,96,100,6.64,GL,1702323897
8,8,lebu,-37.6167,-73.6500,288.93,68,100,4.07,CL,1702323897
9,9,tashtyp,52.7939,89.8983,244.83,99,100,0.57,RU,1702323898
10,10,ust-nera,64.5667,143.2000,253.89,98,100,3.61,RU,1702323898
...,...,...,...,...,...,...,...,...,...,...
553,553,nagqu,31.5000,92.0000,256.85,49,74,1.57,CN,1702323723
555,555,randazzo,37.8774,14.9501,285.21,45,10,1.86,IT,1702324000
557,557,vernon,50.2581,-119.2691,277.38,91,79,0.00,CA,1702324001
558,558,de aar,-30.6497,24.0123,292.65,52,4,2.18,ZA,1702324001


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
try:
    hotel_data = {"City": ideal_weather_df_complete["City"],
                 "Country": ideal_weather_df_complete["Country"],
                 "Coordinates_Lat": ideal_weather_df_complete["Lat"],
                 "Coordinates_Lng": ideal_weather_df_complete["Lng"],
                 "Humdity": ideal_weather_df_complete["Humidity"]}
except:
    pass

hotel_df = pd.DataFrame(hotel_data).copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Coordinates_Lat,Coordinates_Lng,Humdity,Hotel Name
4,reggane,DZ,26.7158,0.1714,30,
6,qaqortoq,GL,60.7167,-46.0333,96,
8,lebu,CL,-37.6167,-73.6500,68,
9,tashtyp,RU,52.7939,89.8983,99,
10,ust-nera,RU,64.5667,143.2000,98,
...,...,...,...,...,...,...
553,nagqu,CN,31.5000,92.0000,49,
555,randazzo,IT,37.8774,14.9501,45,
557,vernon,CA,50.2581,-119.2691,91,
558,de aar,ZA,-30.6497,24.0123,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = ["Coordinates_Lat"]
    lng = ["Coordinates_Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat}{lng}{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
reggane - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
tashtyp - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
baotou - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
thompson - nearest hotel: No hotel found
yermentau - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
laojunmiao - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
des moines - nearest hotel: No hotel found
marg

kokpekty - nearest hotel: No hotel found
aksu - nearest hotel: No hotel found
bodaybo - nearest hotel: No hotel found
muroto-misakicho - nearest hotel: No hotel found
sungho 1-tong - nearest hotel: No hotel found
winnemucca - nearest hotel: No hotel found
borogontsy - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
kula - nearest hotel: No hotel found
zeliezovce - nearest hotel: No hotel found
chernomorskoye - nearest hotel: No hotel found
murghob - nearest hotel: No hotel found
yarmouth - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
east ballina - nearest hotel: No hotel found
gelemso - nearest hotel: No hotel found
northam - nearest hotel: No hotel found
petawawa - nearest hotel: No hotel found
iringa - nearest hotel: No hotel found
tessaoua - nearest hotel: No hotel found
lata - nearest hotel: No hotel found
saldanha - nearest hotel: No hotel found
constantia - nearest hotel: No hotel found
kamuli - nearest hotel: No hotel found

,City,Country,Coordinates_Lat,Coordinates_Lng,Humdity,Hotel Name
4,reggane,DZ,26.7158,0.1714,30,No hotel found
6,qaqortoq,GL,60.7167,-46.0333,96,No hotel found
8,lebu,CL,-37.6167,-73.6500,68,No hotel found
9,tashtyp,RU,52.7939,89.8983,99,No hotel found
10,ust-nera,RU,64.5667,143.2000,98,No hotel found
...,...,...,...,...,...,...
553,nagqu,CN,31.5000,92.0000,49,No hotel found
555,randazzo,IT,37.8774,14.9501,45,No hotel found
557,vernon,CA,50.2581,-119.2691,91,No hotel found
558,de aar,ZA,-30.6497,24.0123,52,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE